In [9]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd
import re

In [19]:
def browser_wait(driver, element, locator=By.XPATH, timeout=10):
    return WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((locator, element))
    )

def scroll(driver, start=0, end='document.body.scrollHeight'):
    driver.execute_script(f'window.scrollTo({start}, {end});')    

In [8]:
# env
USERNAME = 'fahri.lafa@gmail.com'
PASSWORD = '1qw23er4'

In [28]:
# specify chrome options
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
browser = webdriver.Chrome(options=options)

# open the site
print('Opening site...')
browser.get('https://linkedin.com')

# move to login page
try:    
    login_btn = browser_wait(browser, '/html/body/nav/div/a[2]')
    login_btn.click()
except:
    print('Cannot find login button!')

# fill credentials
print('Attempting to login...')
try:
    email_field = browser_wait(browser, '#username', locator=By.CSS_SELECTOR)
    pwd_field = browser_wait(browser, '#password', locator=By.CSS_SELECTOR)
    # insert email password
    email_field.send_keys(USERNAME)
    pwd_field.send_keys(PASSWORD)
    # submit
    time.sleep(1.5)
    pwd_field.send_keys(Keys.RETURN)
    print('Submit login...')
except:
    print('Cannot find email or passowrd field!')

# go to My Network page
print('Move to `My Network` page')
browser.get('https://www.linkedin.com/mynetwork/invite-connect/connections/')

# scroll to bottom
last_height = browser.execute_script('return document.body.scrollHeight')
wait_scroll = 5
while True:
    print('Scrolling...')
    # scroll until the last record
    scroll(browser)
    print('Wait 5s...')
    time.sleep(5)
    new_height = browser.execute_script('return document.body.scrollHeight')
    # if scroll reach the end
    if new_height == last_height:
        try:
            # look for load_more button
            print('Looking for load more button...')
            # load_more = browser_wait(browser, '/html/body/div[7]/div[3]/div/div/div/div/div[2]/div/div/main/div/section/div[2]/div[2]/div/button')
            load_more = browser_wait(browser, 'scaffold-finite-scroll__load-button', locator=By.CLASS_NAME)
            load_more.click()
            print('Wait 5s...')
            time.sleep(5)
            # scroll again
            print('Scrolling...')
            scroll(browser)
        except:
            # no more data
            print('Load more button not found! It seems we are reached the end')
            break
    # update last height
    print('Update last height')
    last_height = new_height
            
# get raw html
print('Get output...')
my_networks = BeautifulSoup(browser.page_source.encode('utf-8'), 'html')
my_networks.prettify()

# write html to file
print('Write output...')
with open('my_networks.html', 'w', encoding='utf-8') as w:
    w.write(str(my_networks))

# anaylzing html tags
print('Analyzing html tags...')



# quit browser
print('Done. Quitting...')
time.sleep(5)
browser.quit()

Opening site...
Attempting to login...
Submit login...
Move to `My Network` page
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Looking for load more button...
Wait 5s...
Scrolling...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scr

In [34]:
containers = my_networks.findAll('li', {'class': 'mn-connection-card artdeco-list'})
len(containers)

840

In [67]:
# parsing profiles list
profiles = []

for c in containers:
    # data model
    m_profile = {
        'name': None,
        'occupation': None,
        'profile_link': 'https://linkedin.com{}',
        'connected_at': None
    }
    m_profile['name'] = c.find('span', {'class': 'mn-connection-card__name'}).text.strip()
    m_profile['occupation'] = c.find('span', {'class': 'mn-connection-card__occupation'}).text.strip()
    m_profile['profile_link'] = m_profile['profile_link'].format(c.find('a', {'class': 'mn-connection-card__link'})['href'])
    m_profile['connected_at'] = c.find('time').text.strip().split('\n')[-1].strip()
    profiles.append(m_profile)    

df_profile = pd.DataFrame(profiles)
df_profile.head()

,name,occupation,profile_link,connected_at
0,"Sie, Evan Setiawan",Class Coordinator - Sekolah Data PACMANN,https://linkedin.com/in/sie-evan-setiawan/,1 hour ago
1,Erika Dwi Puspitasari,Student at Airlangga University,https://linkedin.com/in/erika-dwi-puspitasari-...,1 day ago
2,Ekaterina Shlenova,IT Recruiter in Yandex,https://linkedin.com/in/ekaterina-shlenova-714...,1 day ago
3,eef hikmat,Student at sekolah tinggi ilmu ekonomi 66,https://linkedin.com/in/eef-hikmat-13332a1a6/,1 week ago
4,Amna Maharani,IT Student at Fachhochschule Kiel,https://linkedin.com/in/amaharani/,1 week ago
